In [1]:
!pip install sklearn_crfsuite
!pip uninstall -y scipy numpy tensorflow tensorflow-addons
!pip install numpy==1.23.5
!pip install scipy==1.10.1
!pip install tensorflow==2.12.0
!pip install tensorflow-addons==0.20.0
!pip install pandas
!pip install seqeval
!pip install pyarabic camel_tools
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 98.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.11.0 requires scipy>=1.6.0, which is not installed.
fastai 2.7.19 requires scipy, which is not installed.
jax 0.5.2 requires scipy>=1.11.1, which is not installed.
scikit-learn 1.6.1 requires scipy>=1.6.0, which is not installed.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, which is not installed.
scikit-image 0.25.2 requires scipy>=1.11.4, which is not installed.
jaxlib 0.5.1 requires scipy>=1.11.1, which is not installed.
thinc 8.3.6 requi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 44.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
INFO: pip is looking at multiple versions of jax to determine w

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from seqeval.metrics import classification_report, f1_score, accuracy_score
import tensorflow_addons as tfa
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset
import pyarabic.araby as araby
from camel_tools.utils.normalize import normalize_unicode

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Lecture puis conversion du fichieer en format .csv pour faciliter les manipulations


In [4]:
path= '/content/drive/MyDrive/Datasets/ANERCorp.xlsx'
csv_path = "/content/drive/MyDrive/Datasets/ANERCorp.csv"
df = pd.read_excel(path, header=None)
df.to_csv(csv_path, index=False)
data=pd.read_csv(csv_path, header=None)
data.head(10)

,0,1,2
0,0,1,2
1,فرانكفورت,B-LOC,NaN
2,(د,O,NaN
3,ب,O,NaN
4,أ),O,NaN
5,أعلن,O,NaN
6,اتحاد,B-ORG,NaN
7,صناعة,I-ORG,NaN
8,السيارات,I-ORG,NaN
9,في,O,NaN


Renomage des headers en text/label puis suppression de l'ancien header (0/1)


In [5]:
data = df.drop(2, axis=1)
data.columns = ['text', 'label']
data.to_csv(csv_path, index=False)
data.head()

,text,label
0,فرانكفورت,B-LOC
1,(د,O
2,ب,O
3,أ),O
4,أعلن,O


In [6]:
count= data["label"].value_counts()
print(count)

label
O                 132029
B-LOC               4429
B-PERS              3598
I-PERS              2831
B-ORG               2023
I-ORG               1379
B-MISC              1115
I-LOC                604
I-MISC               539
o                      4
I-PRG                  2
B-ERS                  2
B-ORF                  1
فيرسلاين               1
IPERS                  1
b-misc                 1
‏                      1
B-LOCI-PERS            1
B-OEG                  1
i-misc                 1
Oالإصابة               1
ونايفة</title>         1
B-PERs                 1
B-PRG                  1
B-MSIC                 1
Name: count, dtype: int64


Calcule des classes distinctes ainsii que leur affichage pour mieux connaitre notre dataset

In [7]:
classes = set(data['label'])
print("Le nombre de classe est :", {len(classes)})
classes

Le nombre de classe est : {26}


{'B-ERS',
 'B-LOC',
 'B-LOCI-PERS',
 'B-MISC',
 'B-MSIC',
 'B-OEG',
 'B-ORF',
 'B-ORG',
 'B-PERS',
 'B-PERs',
 'B-PRG',
 'I-LOC',
 'I-MISC',
 'I-ORG',
 'I-PERS',
 'I-PRG',
 'IPERS',
 'O',
 'Oالإصابة',
 'b-misc',
 'i-misc',
 nan,
 'o',
 'فيرسلاين',
 'ونايفة</title>',
 '\u200f'}

In [ ]:
def prepare_data(data, word_col='text', label_col='label'):
    sequences = []
    words = []
    tags = []

    for index, row in data.iterrows():
        if pd.isna(row[word_col]) or pd.isna(row[label_col]):
            if words and tags:
                sequences.append((words, tags))
                words = []
                tags = []
        else:
            words.append(str(row[word_col]))
            tags.append(str(row[label_col]))

    if words and tags:
        sequences.append((words, tags))

    return sequences

In [ ]:
def normalize_arabic_text(text):

    if not isinstance(text, str):
        return text

    text = normalize_unicode(text)

    text = araby.strip_tashkeel(text)

    text = araby.normalize_alef(text)

    text = araby.normalize_hamza(text)

    text = araby.normalize_ligature(text)

    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [10]:
def get_word_features(word):
    return {
        'word.isdigit()': word.isdigit(),
        'word.is_year': bool(re.match(r'^[0-9]{4}$', word)),
        'word.length': len(word),
        'word.prefix1': word[:1] if len(word) > 0 else '',
        'word.prefix2': word[:2] if len(word) > 1 else '',
        'word.suffix1': word[-1:] if len(word) > 0 else '',
        'word.suffix2': word[-2:] if len(word) > 1 else '',
        'word.has_digit': any(c.isdigit() for c in word),
        'word.has_punctuation': any(not c.isalnum() for c in word),
        'word.has_alef': 'ا' in word,
        'word.has_waw': 'و' in word,
        'word.has_ya': 'ي' in word,
        'word.has_ta_marbuta': 'ة' in word,
        'word.has_alif_lam': 'ال' in word,
    }

In [ ]:
def get_contextual_features(words, i):
    features = {}

    if i > 0:
        prev_word = normalize_arabic_text(words[i-1])
        features['prev_word'] = prev_word
        if len(prev_word) >= 2:
            features['prev_word.suffix2'] = prev_word[-2:]

    if i < len(words) - 1:
        next_word = normalize_arabic_text(words[i+1])
        features['next_word'] = next_word
        if len(next_word) >= 2:
            features['next_word.prefix2'] = next_word[:2]

    return features


In [ ]:
def extract_features(words, i):
    word = words[i]
    normalized_word = normalize_arabic_text(word)

    features = {}
    features.update(get_word_features(normalized_word))
    features.update(get_contextual_features(words, i))

    return features

In [13]:
def prepare_features(sequences):
    X, y = [], []

    for words, tags in sequences:
        X_sent, y_sent = [], []
        for i in range(len(words)):
            X_sent.append(extract_features(words, i))
            y_sent.append(tags[i])
        X.append(X_sent)
        y.append(y_sent)

    return X, y

In [ ]:
def train_crf_model(X_train, y_train, params=None):

    if params is None:
        params = {
            'algorithm': 'lbfgs',
            'c1': 0.1,  
            'c2': 0.1, 
            'max_iterations': 100,
            'all_possible_transitions': True
        }

    print("Début de l'entraînement du modèle CRF...")
    print(f"Paramètres: {params}")
    start_time = time.time()

    crf = sklearn_crfsuite.CRF(
        algorithm=params['algorithm'],
        c1=params['c1'],
        c2=params['c2'],
        max_iterations=params['max_iterations'],
        all_possible_transitions=params['all_possible_transitions'],
        verbose=True
    )

    crf.fit(X_train, y_train)

    end_time = time.time()
    training_time = end_time - start_time
    print(f"Entraînement terminé en {training_time:.2f} secondes")

    return crf, training_time


In [ ]:
sequences = prepare_data(data)
train_sequences, test_sequences = train_test_split(sequences, test_size=0.3, random_state=42)

X_train, y_train = prepare_features(train_sequences)
X_test, y_test = prepare_features(test_sequences)

In [ ]:
    crf, training_time = train_crf_model(X_train, y_train)

    import pickle

    with open('/content/drive/MyDrive/modele_CRF_NER.pkl', 'wb') as f:
        pickle.dump(crf, f)

Début de l'entraînement du modèle CRF...
Paramètres: {'algorithm': 'lbfgs', 'c1': 0.1, 'c2': 0.1, 'max_iterations': 100, 'all_possible_transitions': True}


loading training data to CRFsuite: 100%|██████████| 1204/1204 [00:01<00:00, 1079.49it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 61278
Seconds required: 0.239

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.28  loss=90765.68 active=61003 feature_norm=1.00
Iter 2   time=0.14  loss=88139.95 active=59165 feature_norm=0.99
Iter 3   time=0.28  loss=68795.13 active=48330 feature_norm=1.78
Iter 4   time=0.14  loss=64198.67 active=59284 feature_norm=1.80
Iter 5   time=0.14  loss=59277.80 active=60036 feature_norm=1.83
Iter 6   time=0.41  loss=55945.70 active=52993 feature_norm=1.91
Iter 7   time=0.15  loss=53396.17 active=53109 feature_norm=2.28
Iter 8   time=0.29  loss=51539.35 active=52364 feature_norm=2.48
Iter 9   time=0.14  loss=50137.26 active=52376 feature_norm=3.02
Iter 10  time=

In [ ]:
from sklearn.metrics import classification_report
def evaluate_model(crf, X_test, y_test):
    print("Évaluation du modèle...")

    start_time = time.time()
    y_pred = crf.predict(X_test)
    inference_time = time.time() - start_time
    print(f"Temps d'inférence: {inference_time:.4f} secondes")

    
    labels = sorted(set([tag for seq in y_test for tag in seq if tag != 'O']))

    results = {
        'accuracy': metrics.flat_accuracy_score(y_test, y_pred),
        'precision_micro': metrics.flat_precision_score(y_test, y_pred, average='micro', labels=labels),

        'classification_report': classification_report([tag for seq in y_test for tag in seq],
                                                     [tag for seq in y_pred for tag in seq]),
        'inference_time': inference_time
    }

    print("--- Classification Report ---")
    print(results['classification_report'])

    print("\n--- Summary Metrics ---")
    print(f"Accuracy: {results['accuracy']:.4f}")
    print(f"Precision (micro): {results['precision_micro']:.4f}")

    return results, y_pred
results, y_pred= evaluate_model(crf, X_test, y_test)


Évaluation du modèle...
Temps d'inférence: 0.5130 secondes


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


--- Classification Report ---
                precision    recall  f1-score   support

         B-LOC       0.76      0.54      0.63      1591
   B-LOCI-PERS       0.00      0.00      0.00         1
        B-MISC       0.72      0.31      0.43       413
         B-OEG       0.00      0.00      0.00         1
         B-ORG       0.73      0.43      0.54       712
        B-PERS       0.70      0.47      0.57      1289
        B-PERs       0.00      0.00      0.00         1
         B-PRG       0.00      0.00      0.00         1
         I-LOC       0.81      0.49      0.61       261
        I-MISC       0.68      0.24      0.35       174
         I-ORG       0.76      0.53      0.62       446
        I-PERS       0.78      0.60      0.68      1045
         IPERS       0.00      0.00      0.00         1
             O       0.95      0.99      0.97     50319
      Oالإصابة       0.00      0.00      0.00         1
        b-misc       0.00      0.00      0.00         1
        i-misc   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import joblib
def prepare_sentence_features(sentence):
    words = sentence.split()
    X_sent = []

    for i in range(len(words)):
        X_sent.append(extract_features(words, i))

    return X_sent

def predict_entities(model_path, sentence):
    crf = joblib.load(model_path)

    words = sentence.split()

    X_sent = prepare_sentence_features(sentence)

    y_pred = crf.predict([X_sent])[0]

    tagged_sentence = list(zip(words, y_pred))

    return tagged_sentence

def extract_named_entities(tagged_sentence):
    entities = {}
    current_entity = []
    current_tag = None

    for word, tag in tagged_sentence:
        if tag.startswith('B-'):
            if current_entity:
                entity_text = ' '.join(current_entity)
                entity_type = current_tag[2:]

                if entity_type not in entities:
                    entities[entity_type] = []
                entities[entity_type].append(entity_text)

                current_entity = []

            current_entity.append(word)
            current_tag = tag

        elif tag.startswith('I-') and current_entity and tag[2:] == current_tag[2:]:
            current_entity.append(word)

        else:  
            if current_entity:
                entity_text = ' '.join(current_entity)
                entity_type = current_tag[2:] 

                if entity_type not in entities:
                    entities[entity_type] = []
                entities[entity_type].append(entity_text)

                current_entity = []
                current_tag = None

            if tag.startswith('B-'):  
                current_entity.append(word)
                current_tag = tag

    if current_entity:
        entity_text = ' '.join(current_entity)
        entity_type = current_tag[2:] 

        if entity_type not in entities:
            entities[entity_type] = []
        entities[entity_type].append(entity_text)

    return entities

def test_model_on_sentence(model_path, sentence):
    tagged_sentence = predict_entities(model_path, sentence)

    entities = extract_named_entities(tagged_sentence)

    print("=== Résultats de la reconnaissance d'entités nommées ===")
    print(f"Phrase: {sentence}")
    print("\nEntités identifiées:")

    for entity_type, entity_list in entities.items():
        print(f"  {entity_type}:")
        for entity in entity_list:
            print(f"    - {entity}")

    print("\nPhrase étiquetée:")
    for word, tag in tagged_sentence:
        print(f"{word} -> {tag}")

    return entities, tagged_sentence

model_path = "/content/drive/MyDrive/modele_CRF_NER.pkl"

sentence ="زار محمد علي مكاتب شركة جوجل في القاهرة أمس"

entities, tagged_sentence = test_model_on_sentence(model_path, sentence)

=== Résultats de la reconnaissance d'entités nommées ===
Phrase: زار محمد علي مكاتب شركة جوجل في القاهرة أمس

Entités identifiées:
  PERS:
    - محمد
  ORG:
    - جوجل

Phrase étiquetée:
زار -> O
محمد -> B-PERS
علي -> O
مكاتب -> O
شركة -> O
جوجل -> B-ORG
في -> O
القاهرة -> O
أمس -> O


In [ ]:
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 32
LSTM_UNITS = 100          
BATCH_SIZE = 10
EPOCHS = 1
def preprocess_data(df):
    df = df.dropna(subset=['text', 'label'])

    texts = df['text'].tolist()
    labels = df['label'].tolist()

    texts_tokenized = [str(text).split() for text in texts]
    labels_tokenized = [str(label).split() for label in labels]

    words = set([word for text in texts_tokenized for word in text])
    labels_set = set([label for label_seq in labels_tokenized for label in label_seq])

    word_to_idx = {word: idx + 1 for idx, word in enumerate(words)} 
    label_to_idx = {label: idx for idx, label in enumerate(labels_set)}

    X = [[word_to_idx.get(word, 0) for word in text] for text in texts_tokenized]
    y = [[label_to_idx.get(label, 0) for label in label_seq] for label_seq in labels_tokenized]

    X_padded = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post', dtype='int32')
    y_padded = pad_sequences(y, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post', dtype='int32')


    num_labels = len(label_to_idx)
    y_one_hot = np.array([to_categorical(label_seq, num_classes=num_labels) for label_seq in y_padded], dtype='float16')

    return X_padded, y_one_hot, word_to_idx, label_to_idx


In [ ]:
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed
from tensorflow.keras.metrics import Precision, Recall
def create_bilstm_crf_model(vocab_size, num_labels):
    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
    model = Embedding(input_dim=len(word_to_idx) + 1, output_dim=32, input_length=MAX_SEQUENCE_LENGTH, mask_zero=True)(input_layer)
    model = Bidirectional(LSTM(units=64, return_sequences=True,dropout=0.4, recurrent_dropout=0.2))(model)

    output = TimeDistributed(Dense(25, activation="softmax"))(model)

    model = Model(inputs=input_layer, outputs=output)

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=['accuracy',  Precision(), Recall()]
    )
    return model

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
def evaluate_model(model, X_test, y_test, idx_to_label):
    y_pred = model.predict(X_test)
    y_pred_argmax = np.argmax(y_pred, axis=-1)
    y_test_argmax = np.argmax(y_test, axis=-1)

    y_pred_flat = []
    y_test_flat = []

    for i in range(len(y_test_argmax)):
        for j in range(len(y_test_argmax[i])):
           
            if y_test_argmax[i][j] != 0:  
                y_test_flat.append(idx_to_label[y_test_argmax[i][j]])
                y_pred_flat.append(idx_to_label[y_pred_argmax[i][j]])

    report = classification_report(y_test_flat, y_pred_flat, zero_division=0)
    f1 = f1_score(y_test_flat, y_pred_flat, average='weighted', zero_division=0)
    acc= accuracy_score(y_test_flat,y_pred_flat)

    return report, f1, acc

In [ ]:
X, y, word_to_idx, label_to_idx = preprocess_data(data)
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [ ]:
vocab_size = len(word_to_idx)
num_labels = len(label_to_idx)
model = create_bilstm_crf_model(vocab_size, num_labels)

model.summary()

history = model.fit(X_train, y_train,
                       batch_size=BATCH_SIZE,
                       epochs=EPOCHS,
                       validation_split=0.1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 32)           1062560   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 128)         49664     
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 100, 25)          3225      
 ibuted)                                                         
                                                                 
Total params: 1,115,449
Trainable params: 1,115,449
Non-trainable params: 0
_________________________________________________________________
10697/10697 [==============================] - 2857

In [ ]:
report, f1 , acc= evaluate_model(model, X_test, y_test, idx_to_label)
print("Rapport de classification :")
print(report)
print(f"Score F1 : {f1}")
print(f"Accuracy : {acc}")

model.save("/content/drive/MyDrive/modele_BILSTM_NER")

929/929 [==============================] - 35s 37ms/step
Rapport de classification :
              precision    recall  f1-score   support

       B-ERS       0.00      0.00      0.00         1
       B-LOC       0.86      0.73      0.79       825
      B-MISC       0.00      0.00      0.00         0
       B-ORG       0.82      0.44      0.57       430
      B-PERS       0.64      0.57      0.60       711
       B-PRG       0.00      0.00      0.00         1
       I-LOC       0.51      0.43      0.46       103
      I-MISC       0.00      0.00      0.00       111
       I-ORG       0.65      0.08      0.15       267
      I-PERS       0.57      0.16      0.24       546
       I-PRG       0.00      0.00      0.00         1
           O       0.95      1.00      0.98     26482
    Oالإصابة       0.00      0.00      0.00         1
      i-misc       0.00      0.00      0.00         1

    accuracy                           0.94     29480
   macro avg       0.36      0.24      0.27     2

In [28]:
import pickle

with open('/content/drive/MyDrive/word_to_idx.pkl', 'wb') as f:
    pickle.dump(word_to_idx, f)

with open('/content/drive/MyDrive/idx_to_label.pkl', 'wb') as f:
    pickle.dump(idx_to_label, f)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

def predict_for_new_text(new_text, model_path, word_to_idx, label_to_idx, max_seq_length=100):
    loaded_model = load_model(model_path)

    tokens = str(new_text).split()

    sequence = [word_to_idx.get(word, 0) for word in tokens] 

    padded_sequence = pad_sequences([sequence], maxlen=max_seq_length, padding='post', truncating='post', dtype='int32')

    predictions = loaded_model.predict(padded_sequence)

    idx_to_label = {idx: label for label, idx in label_to_idx.items()}
    predicted_indices = np.argmax(predictions, axis=-1)

    result = []
    for i, idx in enumerate(predicted_indices[0]):
        if i >= len(tokens):  
            break
        result.append(idx_to_label[idx])

    return result

     new_phrase ="ذهب أحمد إلى القاهرة"
model_path = "/content/drive/MyDrive/modele_BILSTM_NER"
prediction = predict_for_new_text(new_phrase, model_path, word_to_idx, label_to_idx)
print(f"Étiquettes prédites: {prediction}")

1/1 [==============================] - 1s 538ms/step
Étiquettes prédites: ['O', 'B-PERS', 'O', 'B-LOC']
